In [2]:
using JuMP, Gurobi, DataFrames, CSV

# Housing

In [128]:
data = CSV.read("housing.csv",header = false);

In [129]:
X = Matrix(data[:,1:end-1])
y = data[:,end];
n,p = size(X)

train_n = round(Int, 0.5 * n)
val_n = round(Int, 0.75 * n)

# Training set X, y
y_train = y[1:train_n - 1]
X_train = X[1:train_n - 1, :]
# Cross Validation set X, y
y_validation = y[train_n:val_n - 1]
X_validation = X[train_n:val_n - 1, :]
# Test set X, y
y_test = y[val_n:end];
X_test = X[val_n:end, :]
n, p = size(X_train)
print(n)

252

In [4]:
function reg_outlier(X,y,K)
    n, p = size(X)
    m = Model(solver = GurobiSolver(OutputFlag=0))
    M = 1000

    @variable(m, β[1:p]);
    @variable(m, z[1:p], Bin);
    @constraint(m, sum(z[j] for j = 1:p) <= K);
    @constraint(m, m_gt[j=1:p], β[j] <= M * z[j]);
    @constraint(m, m_lt[j=1:p], -M * z[j] <= β[j]);
   
    a = 0
    for i = 1:n
        a += 0.5(y[i] - dot(β, vec(X_train[i,:])))^2
    end
    @objective(m,Min,a)
    solve(m)
    return(getobjectivevalue(m),getvalue(β))
end

reg_outlier (generic function with 1 method)

In [15]:
function det_outlier(X,y,K,β,M)
    n, p = size(X)
    m = Model(solver = GurobiSolver(OutputFlag=0))
    
    @variable(m, ϵ[1:n]>=0);
    @variable(m, t[1:n], Bin);
    
    @constraint(m, sum(t[i] for i = 1:n) <= K);
    
    for i = 1:n
        @constraint(m, 0.5*(y[i] - dot(β, vec(X_train[i,:])))^2 - M*t[i] <= ϵ[i]);
        @constraint(m, -M*(1-t[i]) <= ϵ[i])
        @constraint(m, ϵ[i] <= M*(1-t[i]))
    end
    
    @objective(m,Min,sum(ϵ[i] for i=1:n))
    solve(m)
    return(getobjectivevalue(m),getvalue(t))
end

det_outlier (generic function with 1 method)

In [132]:
X_train1 = X_train
y_train1 = y_train
z1 = zeros(length(y_train))
err = length(y_train)
iter = 1
while err > 5 | iter < 15
    objective, β = reg_outlier(X_train1,y_train1,10)
    objective, z = det_outlier(X_train,y_train,20,β,1000)
    err = sum(abs.(z1 - z))
    z1 = z
    println("error: ",err)
    X_train1 = X_train[z.==0,:]
    y_train1 = y_train[z.==0,:]
    iter += 1
    println("Iteration: ", iter)
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only
error: 20.0
Iteration: 2
Academic license - for non-commercial use only
Academic license - for non-commercial use only
error: 22.0
Iteration: 3
Academic license - for non-commercial use only
Academic license - for non-commercial use only
error: 8.0
Iteration: 4
Academic license - for non-commercial use only
Academic license - for non-commercial use only
error: 4.0
Iteration: 5


In [19]:
function p2linear1(X, y, ρ)
    m = Model(solver=GurobiSolver(OutputFlag=0))
    p = size(X, 2)
    @variable(m, t)
    @variable(m, β[1:p])
    @variable(m, a[1:p])
    @constraint(m, norm(y - X * β) <= t)
    for i = 1:p
        @constraint(m, β[i] <= a[i])
        @constraint(m, -β[i] <= a[i])
        @constraint(m, a[i] >= 0)
    end
    @objective(m, Min, t + ρ*sum(a[i] for i = 1:p))
    solve(m)
    return getvalue(β)
end

p2linear1 (generic function with 1 method)

In [11]:
sum(z1)
X_train1 = X_train[z1.==0,:];
y_train1 = y_train[z1.==0,:];
objective, β_outlier = reg_outlier(X_train1,y_train1,10)
objective, β = reg_outlier(X_train,y_train,10)
mse_before = mean((y_test - X_test * p2linear1(X_train, y_train, 1)).^2)
mse_after = mean((y_test - X_test * p2linear1(X_train1, y_train1, 1)).^2)
println("mse before:",mse_before)
println("mse after:",mse_after)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
mse before:26.433378883667917
mse after:22.298276534890046


# Cancer

In [42]:
data = CSV.read("cancer_reg.csv",header = true);
data = dropmissing(data)
X = data[.~[(x in [:target_deathrate, :binnedinc, :geography]) for x in names(data)]]
X = Matrix(X)
y = data[3];

In [43]:
n,p = size(X)

train_n = round(Int, 0.8 * n)

y_train = y[1:train_n - 1]
X_train = X[1:train_n - 1, :]

y_test = y[train_n:end];
X_test = X[train_n:end, :]
n, p = size(X_train)
println("obs: ",n)
println("Dimension: ",p)

obs: 472
Dimension: 30


In [122]:
X_train1 = X_train
y_train1 = y_train
z1 = zeros(length(y_train))
err = length(y_train)
iter = 1
M = 1000
while err > 5 | iter < 15
    objective, β = reg_outlier(X_train1,y_train1,25)
    objective, z = det_outlier(X_train,y_train,40,β,M)
    err = sum(abs.(z1 - z))
    z1 = z
    println("error: ",err)
    X_train1 = X_train[z.==0,:]
    y_train1 = y_train[z.==0,:]
    iter += 1
    println("Iteration: ", iter)
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only
error: 0.0
Iteration: 2


In [46]:
println("Total outlier: ",sum(z1))
X_train1 = X_train[z1.==0,:];
y_train1 = y_train[z1.==0,:];
mse_before = mean((y_test - X_test * p2linear1(X_train, y_train, 1)).^2)
mse_after = mean((y_test - X_test * p2linear1(X_train1, y_train1, 1)).^2)
println("mse before:",mse_before)
println("mse after:",mse_after)

Total outlier: 40.0
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
mse before:573.8465763838767
mse after:519.3665784623928


# Crime Data

In [152]:
crime = CSV.read("communities-and-crime.csv", header = false)
X = Matrix(crime[1:end - 1])
y = crime[end];

In [153]:
n,p = size(X)

train_n = round(Int, 0.8 * n)

y_train = y[1:train_n - 1]
X_train = X[1:train_n - 1, :]

y_test = y[train_n:end];
X_test = X[train_n:end, :]
n, p = size(X_train)
println("obs: ",n)
println("Dimension: ",p)

obs: 97
Dimension: 122


In [154]:
function det_out_spe(X,y,K,β,M)
    n, p = size(X)
    m = Model(solver = GurobiSolver(OutputFlag=0))
    
    @variable(m, ϵ[1:n]>=0);
    @variable(m, t[1:n], Bin);
    
    @constraint(m, sum(t[i] for i = 1:n) == K);
    
    for i = 1:n
        @constraint(m, 0.5*(y[i] - dot(β, vec(X_train[i,:])))^2 - M*t[i] <= ϵ[i]);
        @constraint(m, -M*(1-t[i]) <= ϵ[i])
        @constraint(m, ϵ[i] <= M*(1-t[i]))
    end
    
    @objective(m,Min,sum(ϵ[i] for i=1:n))
    solve(m)
    return(getobjectivevalue(m),getvalue(t))
end

det_out_spe (generic function with 1 method)

In [155]:
X_train1 = X_train
y_train1 = y_train
err = length(y_train)
iter = 0
M = 100
objective, β = reg_outlier(X_train1,y_train1,100)
objective, z1 = det_out_spe(X_train,y_train,5,β,M)

Academic license - for non-commercial use only
Academic license - for non-commercial use only


(0.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [156]:
println("Total outlier: ",sum(z1))
X_train1 = X_train[z1.==0,:];
y_train1 = y_train[z1.==0,:];
mse_before = mean((y_test - X_test * p2linear1(X_train, y_train, 1)).^2)
mse_after = mean((y_test - X_test * p2linear1(X_train1, y_train1, 1)).^2)
println("mse before:",mse_before)
println("mse after:",mse_after)

Total outlier: 5.0
Academic license - for non-commercial use only
Academic license - for non-commercial use only
mse before:0.01838653196800799
mse after:0.020140679978177322


# Abalone

In [142]:
data = CSV.read("abalone.csv",header=true)
X = data[.~[(x in [:Whole_weight]) for x in names(data)]]
X = Matrix(X)[:,2:end]
y = data[5];

In [143]:
n,p = size(X)

train_n = round(Int, 0.8 * n)

y_train = y[1:train_n - 1]
X_train = X[1:train_n - 1, :]

y_test = y[train_n:end];
X_test = X[train_n:end, :]
n, p = size(X_train)
println("obs: ",n)
println("Dimension: ",p)

obs: 3337
Dimension: 9


In [ ]:
# X_train1 = X_train
# y_train1 = y_train
# z1 = zeros(length(y_train))
# err = length(y_train)
# iter = 1
# M = 1000000000
# while err > 5 | iter < 15
#     objective, β = reg_outlier(X_train1,y_train1,9)
#     objective, z = det_out_spe(X_train,y_train,50,β,M)
#     err = sum(abs.(z1 - z))
#     z1 = z
#     println("error: ",err)
#     X_train1 = X_train[z.==0,:]
#     y_train1 = y_train[z.==0,:]
#     iter += 1
#     println("Iteration: ", iter)
# end

# Insurance

In [70]:
data = CSV.read("Ready_insurance.csv",header=true)
X = data[.~[(x in [:charges]) for x in names(data)]]
X = Matrix(X)[:,2:end]
y = data[5];

In [71]:
n,p = size(X)

train_n = round(Int, 0.8 * n)

y_train = y[1:train_n - 1]
X_train = X[1:train_n - 1, :]

y_test = y[train_n:end];
X_test = X[train_n:end, :]
n, p = size(X_train)
println("obs: ",n)
println("Dimension: ",p)

obs: 1069
Dimension: 10


In [76]:
X_train1 = X_train
y_train1 = y_train
z1 = zeros(length(y_train))
err = length(y_train)
iter = 1
while err > 5 | iter < 15
    β = p2linear1(X_train, y_train, 1)
    objective, z = det_outlier(X_train,y_train,1,β,100000000000)
    err = sum(abs.(z1 - z))
    z1 = z
    println("error: ",err)
    X_train1 = X_train[z.==0,:]
    y_train1 = y_train[z.==0,:]
    iter += 1
    println("Iteration: ", iter)
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only
error: 1.0
Iteration: 2


In [77]:
println("Total outlier: ",sum(z1))
X_train1 = X_train[z1.==0,:];
y_train1 = y_train[z1.==0,:];
mse_before = mean((y_test - X_test * p2linear1(X_train, y_train, 1)).^2)
mse_after = mean((y_test - X_test * p2linear1(X_train1, y_train1, 1)).^2)
println("mse before:",mse_before)
println("mse after:",mse_after)

Total outlier: 1.0
Academic license - for non-commercial use only
Academic license - for non-commercial use only
mse before:4.182800335779048e7
mse after:4.1916289253386356e7


# House

In [52]:
train = readtable("housetrain.csv")
test = readtable("housetest.csv")
data = [train;test];
X = data[~[(x in [:Median_house_value]) for x in names(data)]]
X = Matrix(X)
y = data[4];

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #readtable#232(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at /Users/wenweiliu/.julia/v0.6/DataFrames/src/deprecated.jl:1045
 [3] readtable(::String) at /Users/wenweiliu/.julia/v0.6/DataFrames/src/deprecated.jl:1045
 [4] include_string(::String, ::String) at ./loading.jl:522
 [5] softscope_include_string(::Module, ::String, ::String) at /Users/wenweiliu/.julia/v0.6/SoftGlobalScope/src/SoftGlobalScope.jl:66
 [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/wenweiliu/.julia/v0.6/IJulia/src/execute_request.jl:193
 [7] (::Compat.#inner#6{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/wenweiliu/.julia/v0.6/Compat/src/Compat.jl:189
 [8] eventloop(::ZMQ.Socke

In [53]:
n,p = size(X)

train_n = round(Int, 0.8 * n)

y_train = y[1:train_n - 1]
X_train = X[1:train_n - 1, :]

y_test = y[train_n:end];
X_test = X[train_n:end, :]
n, p = size(X_train)
println("obs: ",n)
println("Dimension: ",p)

obs: 8483
Dimension: 5


In [66]:
X_train1 = X_train
y_train1 = y_train
z1 = zeros(length(y_train))
err = length(y_train)
iter = 1
while err > 5 | iter < 15
    β = p2linear1(X_train, y_train, 1)
    objective, z = det_outlier(X_train,y_train,10,β,1000000000000)
    err = sum(abs.(z1 - z))
    z1 = z
    println("error: ",err)
    X_train1 = X_train[z.==0,:]
    y_train1 = y_train[z.==0,:]
    iter += 1
    println("Iteration: ", iter)
end

Academic license - for non-commercial use only
Academic license - for non-commercial use only
error: 10.0
Iteration: 2
Academic license - for non-commercial use only
Academic license - for non-commercial use only
error: 0.0
Iteration: 3


In [67]:
println("Total outlier: ",sum(z1))
X_train1 = X_train[z1.==0,:];
y_train1 = y_train[z1.==0,:];
mse_before = mean((y_test - X_test * p2linear1(X_train, y_train, 1)).^2)
mse_after = mean((y_test - X_test * p2linear1(X_train1, y_train1, 1)).^2)
println("mse before:",mse_before)
println("mse after:",mse_after)

Total outlier: 10.0
Academic license - for non-commercial use only
Academic license - for non-commercial use only
mse before:13664.100561203422
mse after:13661.131550889013
